<a href="https://colab.research.google.com/github/pahdsn/ati_sense_hackathon/blob/master/Harrycode_pah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Installations

#!pip install geopandas
#!pip install rasterio
!pip install rasterstats

     |████████████████████████████████| 133kB 6.8MB/s 


In [2]:
# Imports

import numpy as np
from matplotlib import pyplot as plt
import geopandas as gpd
import pandas as pd
import rasterio
import os
import glob
from google.colab import drive
import gdal
from shapely.geometry import Point
from PIL import Image
from tqdm import tqdm
from torch.utils.data import Dataset
from torchvision import transforms
import torch
import torch.nn as nn

import fiona
import rasterio
import rasterio.mask
from rasterio.plot import show


In [132]:
# Mount Drive and set up paths

drive.mount('/content/drive')
#os.chdir('/content/drive/My Drive/Polar_Hack')
os.chdir('/content/drive/My Drive/ati_sense_prac')
SAMPLING_DIR = "./samples/"
META_DIR = "./samples_meta/"
SHAPEFILE_DIR = "./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/" 
TIFF_DIR = "./Sentinel geotiffs/"
subset_dir = SHAPEFILE_DIR+"subset/"

shapefiles = glob.glob(SHAPEFILE_DIR+'*.shp') #[0:2]
images = glob.glob(TIFF_DIR+'*.tif')
shapefiles_subset = glob.glob(subset_dir+'*.shp')
#print(subset_dir+'*.tif')
print(shapefiles_subset)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/*.tif
['./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181113t074529I.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181113t074529W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180213t175444W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180911t175548W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181016t072958W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180612t180423W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181218t075437W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180515t174633W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180116t075430W.shp', './EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/sub

In [28]:
#Just working out how to extract shape from image file
#tiff0 =  gdal.Open(images[0])
#tiff0a = gdal.Open(images[0]).ReadAsArray()
#tiff1a = gdal.Open(images[-1]).ReadAsArray()
#print(tiff0a.shape)
#print(tiff1a.shape)

(3, 14179, 15598)
(3, 15294, 15597)


In [16]:
# Define routine for extracting label on subset of image

def get_id(shapefile):
  '''
  Extracts datetime component of name
  '''
  return shapefile.split("_")[-1][:-4].upper()

def geo_ref(x,y,GT):
  '''
  return georeferenced point from pixel coordinates
  '''
  X_geo = GT[0] + x * GT[1] + y * GT[2]
  Y_geo = GT[3] + x * GT[4] + y * GT[5]
  return Point(X_geo, Y_geo)

def sample(shapefile,x,y,N):
  '''
  Find tiff file, create NxN sample with origin (x,y) in pixel coordinates
  Return id, sample and class from shapefile
  '''
  id = get_id(shapefile)
  shape_data = gpd.read_file(shapefile)
  tiff =  gdal.Open([g for g in images if id in g][0])
  point = geo_ref(x+N/2,y+N/2,tiff.GetGeoTransform())
  i=0
  classification = None
  while i < shape_data.shape[0] and classification == None:
    if shape_data['geometry'][i].contains(point):
      classification = shape_data['poly_type'][i]
    i += 1
  if classification != None:
    im = Image.fromarray(np.transpose(tiff.ReadAsArray()[:,x:x+N,y:y+N],(1,2,0)))
    image_name = SAMPLING_DIR+id+'X'+str(x)+'Y'+str(y)+'.png'
    im.save(image_name)
    return id, classification, image_name
  else:
    return None, None, None

In [17]:
# Raw image dimensions
i = 0 #change this to be the image you want
tiff_as = gdal.Open(images[i]).ReadAsArray()
xx = tiff_as.shape[2] #15564 
yy = tiff_as.shape[1] #15218
print(xx,yy)


def get_samples(grid_space, sample_size, chunk_size):
  '''
  Grid space - how densely to sample the raw S1 images
  sample size - size of square sample images (both in pixels)
  Saves sample images in png format and metadata as csv file
  '''
  metadata = pd.DataFrame(columns=['id','x','y','label','image'])
  for S in tqdm(shapefiles):
    for x in np.arange(2*grid_space,xx-2*grid_space,grid_space):
      for y in np.arange(2*grid_space,yy-2*grid_space,grid_space):
        id, label, name = sample(S,x,y,sample_size)
        if label != None:
          metadata = metadata.append({'id':id,'x':x,'y':y,'label':label,'image':name},ignore_index=True)
    metadata.to_csv(META_DIR+'samples.csv')
  return metadata

15598 14179


In [140]:
#Trying to mask tifs based on masking shapefile shapes
def sample2(i_shp):
  '''
  Find tiff file, create NxN sample with origin (x,y) in pixel coordinates
  Return id, sample and class from shapefile
  '''
  
  with fiona.open(i_shp, "r") as shapefile:
      #shapes = [feature["geometry"] for feature in shapefile]
      shapes = [feature["geometry"] for feature in shapefile]
      #print(shapes) #= shapes['poly_type'=='I']
  with rasterio.open(images[0]) as src:
      ctype = str(i_shp[-5])
      #could also loop through each shape and mask each individ shape
      out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
      if ctype=="I":
        k=1
      elif ctype=="W":
        k=2
      elif ctype=="L":
        k=3
      out_image=k*out_image
      out_meta = src.meta

  return out_image, out_transform, out_meta

In [148]:
# Raw image dimensions
i = 0 #change this to be the image you want
tiff_as = gdal.Open(images[i]).ReadAsArray()
xx = tiff_as.shape[2] #15564 
yy = tiff_as.shape[1] #15218
print(xx,yy)

def get_samples2(shapefiles):
  '''
  Pass it a shapefile_subset and have it return an array of masks w/ classifications
  '''
  out_images=[]
  
  
  metadata = pd.DataFrame(columns=['id','x','y','label','image'])
  
  for S in tqdm(shapefiles):
    pname, fname = os.path.split(S)
    shapefile_subset = glob.glob(subset_dir+'*'+fname+'*.shp')
    out_image_p=[]
    for p in shapefile_subset:
    #for x in np.arange(2*grid_space,xx-2*grid_space,grid_space):
    #  for y in np.arange(2*grid_space,yy-2*grid_space,grid_space):
      shapefile_subset[p]
      out_image, out_transform, out_meta = sample2(shapefile_subset)
    #    if label != None:
      out_image_p = out_image_p + out_image
    #out_images = out_image_p + out_image
    #metadata = metadata.append({'id':id,'x':x,'y':y,'label':label,'image':name},ignore_index=True)
    #metadata.to_csv(META_DIR+'samples.csv')
    out_images=[out_images,out_image_p]
  return out_images

15598 14179


In [149]:
out_images = get_samples2(shapefiles)
out_images



100%|██████████| 12/12 [00:00<00:00, 443.06it/s]


[[[[[[[[[[[[[], []], []], []], []], []], []], []], []], []], []], []], []]

In [20]:
inum = 0 #change this to be the image you wwant to get the xx, xy for
csfn = get_id(shapefiles[inum])
tiff_as = gdal.Open(images[i]).ReadAsArray()
xx = tiff_as.shape[2] #15564 
yy = tiff_as.shape[1] #15218
print(xx,yy)
#point = geo_ref(x,y,GT)
#id, classification, image_name = sample(shapefile,x,y,N)
grid_space = 1000
sample_size = 32
metadata = get_samples(grid_space, sample_size)


  0%|          | 0/2 [00:00<?, ?it/s]

15598 14179



100%|██████████| 2/2 [06:29<00:00, 194.91s/it]


In [105]:
# Raw image dimensions
tiff_as = gdal.Open(images[i]).ReadAsArray()
xxi = tiff_as.shape[2] #15564 
yyi = tiff_as.shape[1] #15218
print(xxi, yyi)
#500pixel spacing 
#sample size 64
#4713 samples

subset_dir = SHAPEFILE_DIR+"subset/"
i_shp = subset_dir + "seaice_s1_20181113t074529I.shp"
ctype = i_shp[-5]
print(ctype)

with fiona.open(i_shp, "r") as shapefile:
    #shapes = [feature["geometry"] for feature in shapefile]
    shapes = [feature["geometry"] for feature in shapefile]
    #print(shapes) #= shapes['poly_type'=='I']
with rasterio.open(images[0]) as src:
    #ctype = i_shp
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
print(out_image)

15598 14179
I
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [101]:
i=0
shape_data = gpd.read_file(shapefiles[i])
pname, fname = os.path.split(shapefiles[i])
print(fname)
print(shapefiles[i])
#shape_data['geometry'][0]
#shape_data['geometry'][1]
#from rasterstats import zonal_stats
#listofzones = zonal_stats(shapefiles[0], images[0],
#            stats="mean")
#listofzones
def subset_by_class(shapefiles, shape_data, ctype):
  '''
  Basically this creates shapefiles w/ just 1 class from each of the origional shapefiles
  '''
  #for ctype in ['I', 'L', 'W']:
  for i in range(0,len(shapefiles)):
    shape_data = gpd.read_file(shapefiles[i])
    pname, fname = os.path.split(shapefiles[i])
    shape_data_s = shape_data.copy()
    shape_data_s=shape_data_s[shape_data_s['poly_type']==ctype]
    #shape_data
    #[feature["geometry"] for feature in shape_data]
    shape_data_s.to_file(SHAPEFILE_DIR+"subset/"+fname[0:-4]+ctype+".shp")
    print(SHAPEFILE_DIR+"subset/"+fname[0:-4]+ctype+".shp")

subset_by_class(shapefiles, shape_data, 'I')

seaice_s1_20181113t074529.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/seaice_s1_20181113t074529.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181113t074529I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180213t175444I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180911t175548I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181016t072958I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180612t180423I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20181218t075437I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180515t174633I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180116t075430I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180417t074606I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180814t075344I.shp
./EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/subset/seaice_s1_20180313t181225

In [16]:

LABELS = {
	"L": 0,
	"W": 1,
	"I": 2,
}
samples = pd.read_csv(META_DIR+'samples.csv',usecols=['id','x','y','label','image'])
samples['label'] = [LABELS.get(ll) for ll in samples['label']]
samples.head()

FileNotFoundError: ignored

In [ ]:
samples.values[:,4]


In [ ]:
# Set up data for torch

TRAIN_SIZE = 0.7

class PolarPatch(Dataset):
    '''
    TorchUtils dataset
    '''
    def __init__(self, transform=None, split="train"):
        super(PolarPatch, self).__init__()

        assert split in ["train", "val"]
        
        # TODO: load in meta data, which should be of shape (3, N) - N being the number of samples
        meta = samples.values

        train_dim = int(TRAIN_SIZE * len(meta))
        
        if split == "train":
            meta = meta[:train_dim]
        else:
            meta = meta[train_dim:]                   
        self.images = meta[:,4]
        self.coords = [(row[1], row[2]) for row in meta]

        # Targets in integer form for computing cross entropy
        self.targets = meta[:,3]
        self.transform = transform


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        x = Image.open(self.images[index]) # change this file format if needed
        y = self.targets[index]
        coord = self.coords[index]

        if self.transform:
        	x = self.transform(x)

        return x, y, coord


In [ ]:
BATCH_SIZE = 128

data_transform = transforms.Compose([
    # TODO: add whatever else you need - normalisation, augmentation, etc.
	transforms.ToTensor(),
])

train_set = PolarPatch(
    split='train',
    transform=data_transform
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

test_set = PolarPatch(
    split='val',
    transform=data_transform
)

test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)



In [ ]:
# nn i torch neural network class
# Create a specific nn for our purposes called PolarNet

class PolarNet(nn.Module): # nn.Module is base class for all networks
    def __init__(self, n_classes=3):
        super(PolarNet, self).__init__()
# Super means inherit attributes, presumably from nn.Module
        self.features = nn.Sequential(
            # TODO: build your own architecture here; one conv layer and ReLU here as an example only
            nn.Conv2d(3, 16, kernel_size=5, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=2),
            nn.Conv2d(16,32,kernel_size=5,padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2),stride=2)
        ) # 13*13 * 32 = 5408 pixels

        self.classifier = nn.Sequential(
            nn.Linear(5408,36),
            nn.ReLU(), 
            nn.Linear(36, 3)
        )      

    def forward(self, x):
        # as an example; alter as needed depending on your architecture
        x = self.features(x)

        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
# Device configuration - defaults to CPU unless GPU is available on device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = PolarNet().to(DEVICE)
loss_fn = nn.CrossEntropyLoss()

# Stochastic Gradient Descent
optimizer = torch.optim.SGD(
	model.parameters(),
	lr=0.001,
	weight_decay=0.0005,
	momentum=0.9,
)

In [ ]:
'''
David Hogg's functions (modified) for capturing the stats during training
'''

def stats(loader, net):
    correct = 0
    total = 0
    running_loss = 0
    n = 0
    with torch.no_grad():
        for data in loader:
            images, labels, coords = data
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = net.forward(images)
            loss = loss_fn(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)    # add in the number of labels in this minibatch
            correct += (predicted == labels).sum().item()  # add in the number of correct labels
            running_loss += loss
            n += 1
    return running_loss/n, correct/total 

def statsplot(statsrec,name):
    fig, ax1 = plt.subplots()
    plt.plot(statsrec[0], 'r', label = 'training loss', )
    plt.plot(statsrec[1], 'g', label = 'test loss' )
    plt.legend(loc='center')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Training and test loss, and test accuracy')
    ax2=ax1.twinx()
    ax2.plot(statsrec[2], 'b', label = 'test accuracy')
    ax2.set_ylabel('accuracy')
    plt.legend(loc='upper left')
    plt.savefig('./figures/'+name+'.png')
    plt.show()
    

## Training

In [ ]:
nepochs = 10
# How many times go through the full training dataset

statsrec = np.zeros((3,nepochs))
# Numy array for holding some results

for epoch in range(nepochs):  # loop over the dataset multiple times

    running_loss = 0.0
    n = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels, coords = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
         # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward, backward, and update parameters
        outputs = model.forward(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
    
        # accumulate loss
        running_loss += loss.item()
        n += 1
    
    ltrn = running_loss/n
    ltst, atst = stats(test_loader, model)
    statsrec[:,epoch] = (ltrn, ltst, atst)
    print(f"epoch: {epoch} training loss: {ltrn: .3f}  test loss: {ltst: .3f} test accuracy: {atst: .1%}")

print('********** Finished Training ***************')

In [ ]:
statsplot(statsrec,'5408_secondrun')